In [ ]:
import RNA
import infrared as ir
import infrared.rna as rna
import random
import math
from collections import Counter
import time
import argparse


In [ ]:
#read in ArchiveII fasta file and store sequences and structures
file = "ArchiveII.fa"
structures = []
sequences = []
with open(file, "r") as f:
    for i, line in enumerate(f):
        if i%3==2:
            structures.append(line[:-1])
        elif i%3==1:
            sequences.append(line[:-1])

<h1> Easy Version 

In [ ]:
'''creates new sequences from structures using infrared in a simple, fast way

This cell creates new sequences from the read-in ArchiveII.fa file. For each sequences
a new infrared model is created, to which the constraints from the structures are added. 
Additionally also the GC content of the structure is added. From this model a new sequence is samples 
and stored in the new_sequences list
'''
new_sequences = []

tik = time.time()
for target, seq in zip(structures, sequences):
    n = len(target)
    model = ir.Model(n, 4)
    print(type(model))
    model.add_constraints(rna.BPComp(i,j) for (i,j) in rna.parse(target))
    model.add_functions([rna.GCCont(i) for i in range(n)], 'gc')
    
    base_count = Counter(seq)
    GC = (base_count["G"]+base_count["C"]) / len(seq)
    model.set_feature_weight(GC, 'gc')
    
    sampler = ir.Sampler(model)
    sample = sampler.sample()
    new_seq = rna.ass_to_seq(sample)
    new_sequences.append(new_seq)

tok = time.time()
calc_time_easy = (tok-tik)/60
print(f"search took {calc_time_easy:.2f} minutes for {len(sequences)} sequences")

<h1> Frequency table

In [ ]:
def single_target_design_model(target):
    '''creates infrared model from target structure
    
    function which takes in a target structure, and creates and infrared model, to which the
    constraints of the target structure are added.
    
    Args:
        target (str): structure to which the model should fit
        
    Returns:
        model (infrared.Model): 
    '''
    n, bps = len(target), rna.parse(target)
    model = ir.Model(n, 4)
    model.add_constraints(rna.BPComp(i, j) for (i, j) in bps)
    model.add_functions([rna.GCCont(i) for i in range(n)], 'gc')
    model.add_functions([rna.BPEnergy(i, j, (i-1, j+1) not in bps) for (i,j) in bps], 'energy')
    #model.set_feature_weight(-1.5, 'energy')
    return model


def target_frequency(sequence, target):
    '''calculated frequency from sequence and target structure
    
    function which takes in a sequence and atarget structure. From these two the frequency
    of the target structure regarding the given sequence is calculated using ViennaRNA
    
    Args:
        target (str): structure to which the model should fit
        target (str): structure to which the model should fit
        
    Returns:
        frequency (float): frequency of target structure of folded sequence
    '''
    fc = RNA.fold_compound(sequence)
    fc.pf()
    return fc.pr_structure(target)

In [ ]:
'''creates new sequences from structures using infrared based on frequency table

This cell creates new sequences from the read-in ArchiveII.fa file. For each sequences
a new infrared is created, to which the constraints from the structures are added using the function single_target_design_model. 
then the best value for the frequency (best) and the best sequence (best_seq) are initilized with 0 and None. Then for 100 iterations
a new sequence is sampled from the model and the frequency is calculated using the function target_frequency. If the frequency is better
as the best-saved one, the frequency is stored under best and the sequence is stored udner best_seq. If after 100 iterations there is best_seq
saved, it is added to the new_sequences list. If no best_seq is saved, the last sample is added to the new_sequences list
'''
new_sequences = []

tik = time.time()
#for target, seq in zip(structures, sequences):
for j, target in enumerate(structures):
    n = len(target)
    sampler = ir.Sampler(single_target_design_model(target))

    best = 0
    best_seq = None
    for i in range(100):
        new_seq = rna.ass_to_seq(sampler.targeted_sample())
        freq = target_frequency(new_seq,target)
        if freq > best:
            best = freq
            best_seq = new_seq
            #print(f"{i} {best_seq} {freq:.6f}")
    if best_seq:
        new_sequences.append(best_seq)
    else:
        new_sequences.append(new_seq)
    if j%10 == 0 and j != 0:
        print(f"{j} of {len(structures)} structures done")
        

tok = time.time()
calc_time_easy = (tok-tik)/60
print(f"search took {calc_time_easy:.2f} minutes for {len(sequences)} sequences") #

<h1> Sampling with constraint generation

In [ ]:
'''creates new sequences from structures using infrared based on constraint generation

This cell creates new sequences from the read-in ArchiveII.fa file. For each sequences
a new infrared is created, to which the constraints from the structures are added using the function single_target_design_model. 
Then for 50 iterations a sequence is searched, where the minimum free energy structure (MFE) is the target structure. If the MFE of the new sequence is the target structure the search stops.
If not the base pairs of the MFE are added to a Counter. After updating the counter in each iteration, the two most common base pairs are added to a list, which is then added to the model
as negative constraint. Then the search repeats ten times or when the constraints are no longer consistent.
If a optimal sequence is found it is added to the new_sequences list, if no sequence was found, nothing is added to the list.
'''

new_sequences = []

tik = time.time()

for j, target in enumerate(structures[5:7]):
    n = len(target)
    bps = rna.parse(target)
    def cg_design_iteration():
        model = single_target_design_model(target)
        model.add_constraints(rna.NotBPComp(i, j) for (i, j) in dbps)
        sampler = ir.Sampler(model, lazy=True)
        if sampler.treewidth() > 10 or not sampler.is_consistent():
            return "Not found"
        ctr = Counter()
        found, sol = False, None
        for i in range(50):
            seq = rna.ass_to_seq(sampler.targeted_sample())
            fc = RNA.fold_compound(seq)
            mfe, mfe_e = fc.mfe()
            if target == mfe:
                sol = seq
                break
            ctr.update(rna.parse(mfe))
        ndbps = [x[0] for x in ctr.most_common() if x[0] not in bps]
        dbps.extend(ndbps[:2])
        return sol
    dbps, seq = [], None
    while seq is None:
        seq = cg_design_iteration()
    if seq:
        new_sequences.append(seq)
    if j%10 == 0 and j != 0:
        print(f"{j} of {len(structures)} structures done")

tok = time.time()
calc_time_easy = (tok-tik)/60
print(f"search took {calc_time_easy:.2f} minutes for 2 sequences") #{len(sequences)}